1.) What is inductive reasoning? Deductive reasoning? Give an example of each, different from the examples given in class.

Inductive reasoning is the creation of general conclusions (or rules) based on specific observations. An example of such reasoning would be concluding that all peanuts are red, because every peanut you've ever seen is red.

Deductive reasoning is the creation of specific conclusions, based on general observations/rules. An example of such reasoning would be concluding that dolphins must have some hair/fur because all dolphins are mammals, and all mammals must have some fur.

2.) Preprocess your dataset. 

In [2]:
import pandas as pd

3.) Create a decision tree model tuned to the best of your abilities. Explain how you tuned it.

4.) Create a random forest model tuned to the best of your abilities. Explain how you tuned it.

5.) Create an xgboost model tuned to the best of your abilities. Explain how you tuned it.